# Starter Notebook

Install and import required libraries

In [1]:
!pip install transformers datasets evaluate accelerate peft trl bitsandbytes
!pip install nvidia-ml-py3
!pip install scikit-learn matplotlib seaborn


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pandas as pd
import torch
from transformers import RobertaModel, RobertaTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, ClassLabel
import pickle

/home/joey/sp25-dl/project2/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Tokenizer and Preprocess Data

In [3]:
base_model = 'roberta-base'

dataset = load_dataset('ag_news', split='train')
tokenizer = RobertaTokenizer.from_pretrained(base_model)

def preprocess(examples):
    tokenized = tokenizer(examples['text'], truncation=True, padding=True)
    return tokenized

tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [4]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


## Make train and eval split

In [ ]:
# Split the original training set
split_datasets = tokenized_dataset.train_test_split(test_size=1920, seed=42)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

print("Number of train samples:", len(train_dataset))
print("Number of eval samples:", len(eval_dataset))

Number of train samples: 118720
Number of eval samples: 1280


## Training Setup

In [6]:
# To track evaluation accuracy during training
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy
    }

In [7]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

def evaluate_model(inference_model, dataset, labelled=True, batch_size=8, data_collator=None):
    """
    Evaluate a PEFT model on a dataset.

    Args:
        inference_model: The model to evaluate.
        dataset: The dataset (Hugging Face Dataset) to run inference on.
        labelled (bool): If True, the dataset includes labels and metrics will be computed.
                         If False, only predictions will be returned.
        batch_size (int): Batch size for inference.
        data_collator: Function to collate batches. If None, the default collate_fn is used.

    Returns:
        If labelled is True, returns a tuple (metrics, predictions)
        If labelled is False, returns the predictions.
    """
    # Create the DataLoader
    eval_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    if labelled:
        metric = evaluate.load('accuracy')

    # Loop over the DataLoader
    for batch in tqdm(eval_dataloader):
        # Move each tensor in the batch to the device
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions.append(predictions.cpu())

        if labelled:
            # Expecting that labels are provided under the "labels" key.
            references = batch["labels"]
            metric.add_batch(
                predictions=predictions.cpu().numpy(),
                references=references.cpu().numpy()
            )

    # Concatenate predictions from all batches
    all_predictions = torch.cat(all_predictions, dim=0)

    if labelled:
        eval_metric = metric.compute()
        print("Evaluation Metric:", eval_metric)
        return eval_metric, all_predictions
    else:
        return all_predictions

## Design Space Exploration

### Config

In [8]:
output_base_dir = "dse_results" # base directory for all DSE runs
os.makedirs(output_base_dir, exist_ok=True)

# hyperparameter ranges for DSE
lora_ranks = [4, 5, 6, 7] 
lora_alpha_scaling = [1, 2, 3, 4]


### Design Space Exploration Loop

In [ ]:
import gc # Garbage collector for potentially clearing GPU memory

results = []

for rank in lora_ranks:
    for alpha_scale in lora_alpha_scaling:
        alpha = rank * alpha_scale
        run_name = f"rank_{rank}_alpha_{alpha}"
        print(f"\n--- Starting Run: {run_name} ---")

        # Define output directory for this specific run
        current_output_dir = os.path.join(output_base_dir, run_name)
        os.makedirs(current_output_dir, exist_ok=True)

        # 1. Load Base Model (Load fresh for each run)
        print("Loading base model...")

        model = RobertaForSequenceClassification.from_pretrained(
            base_model,
            id2label=id2label)

        # Move model to GPU if possible
        if torch.cuda.is_available():
            model.to('cuda')

        # Configure LoRA
        print(f"Configuring LoRA with r={rank}, alpha={alpha}")
        peft_config = LoraConfig(
            r=rank,  # LoRA rank
            lora_alpha=alpha,  # Alpha parameter for scaling
            lora_dropout=0.5, # Dropout probability for LoRA layers
            target_modules=["query", "key", "value"], # Apply LoRA to these layers
            bias="none",  # Don't train bias parameters
            task_type="SEQ_CLS", # Specify the task type
        )

        peft_model = get_peft_model(model, peft_config)

        print("PEFT Model Configured:")
        peft_model.print_trainable_parameters()

        training_args = TrainingArguments(
            output_dir=current_output_dir,
            report_to=None,
            eval_strategy="steps",
            logging_steps=100,
            learning_rate=1e-5,
            max_steps=4000,
            num_train_epochs=1,
            use_cpu=False,
            dataloader_num_workers=4,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64, # or 128
            optim="adamw_torch",
            gradient_checkpointing=False,
            gradient_checkpointing_kwargs={'use_reentrant': True},
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False
        )

        trainer = Trainer(
            model=peft_model,
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            data_collator=data_collator,
        )

        # 6. Train the model
        print("Starting training...")
        try:
            train_result = trainer.train()
            print("Training finished.")
            trainer.save_model()

            # 7. Evaluate the model after training
            print("Evaluating model on evaluation set...")
            eval_metrics, _ = evaluate_model(
                peft_model,
                eval_dataset,
                labelled=True,
                batch_size=training_args.per_device_eval_batch_size,
                data_collator=data_collator
            )
            final_accuracy = eval_metrics.get('accuracy', float('nan'))

        except Exception as e:
            print(f"!!! ERROR during training/evaluation for {run_name}: {e}")
            final_accuracy = float('nan')  # Record failure

        # 8. Store results
        results.append({
            "lora_rank": rank,
            "lora_alpha": alpha,
            "accuracy": final_accuracy,
            "output_dir": current_output_dir
        })
        print(f"Run {run_name} completed. Accuracy: {final_accuracy:.4f}")

        # 9. Clean up memory (Important!)
        del model
        del peft_model
        del trainer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Starting Run: rank_4_alpha_4 ---
Loading base model...


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Configuring LoRA with r=4, alpha=4
PEFT Model Configured:
trainable params: 814,852 || all params: 125,463,560 || trainable%: 0.6495
Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.398700,1.385969,0.247656
200,1.379900,1.375162,0.247656
300,1.372000,1.365314,0.428906
400,1.361000,1.354804,0.612500
500,1.350900,1.341707,0.653906
600,1.328100,1.321327,0.821875
700,1.303400,1.296508,0.790625
800,1.279600,1.261755,0.854688
900,1.237500,1.215027,0.861719
1000,1.178300,1.144910,0.875000


...

Configuring LoRA with r=7, alpha=28
PEFT Model Configured:
trainable params: 980,740 || all params: 125,629,448 || trainable%: 0.7807
Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.393400,1.381527,0.310937
200,1.376600,1.367744,0.342187
300,1.357300,1.336867,0.814844
400,1.277500,1.169184,0.840625
500,0.941500,0.665824,0.868750
600,0.568900,0.452165,0.876563
700,0.440000,0.402947,0.874219
800,0.415100,0.375679,0.885156
900,0.384600,0.364113,0.883594
1000,0.353000,0.355569,0.890625


Training finished.
Evaluating model on evaluation set...


100%|██████████| 20/20 [00:25<00:00,  1.27s/it]

Evaluation Metric: {'accuracy': 0.89296875}
Run rank_7_alpha_28 completed. Accuracy: 0.8930


## Post-DSE Analysis

In [10]:
# Convert results to DataFrame for easy viewing/sorting
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="accuracy", ascending=False)

print(results_df)

# Save results to CSV
results_csv_path = os.path.join(output_base_dir, "dse_summary.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"\nFull DSE results saved to: {results_csv_path}")


    lora_rank  lora_alpha  accuracy                   output_dir
11          6          24  0.893750  dse_results/rank_6_alpha_24
15          7          28  0.892969  dse_results/rank_7_alpha_28
7           5          20  0.889844  dse_results/rank_5_alpha_20
14          7          21  0.889844  dse_results/rank_7_alpha_21
10          6          18  0.889844  dse_results/rank_6_alpha_18
9           6          12  0.889062  dse_results/rank_6_alpha_12
6           5          15  0.888281  dse_results/rank_5_alpha_15
3           4          16  0.886719  dse_results/rank_4_alpha_16
2           4          12  0.885938  dse_results/rank_4_alpha_12
13          7          14  0.885938  dse_results/rank_7_alpha_14
5           5          10  0.885938  dse_results/rank_5_alpha_10
1           4           8  0.885156   dse_results/rank_4_alpha_8
8           6           6  0.884375   dse_results/rank_6_alpha_6
12          7           7  0.883594   dse_results/rank_7_alpha_7
0           4           4

### Run Inference on unlabelled dataset

In [23]:
# Load best model from DSE
model_path = "dse_results/rank_4_alpha_4"

print(f"Loading best model from: {model_path}")
# Load the base model again
base_inference_model = RobertaForSequenceClassification.from_pretrained(
    base_model,
    id2label=id2label,
    num_labels=num_labels
)
# Load the PEFT adapter
inference_model = PeftModel.from_pretrained(base_inference_model, model_path)
inference_model.merge_and_unload() # Optional: Merge adapter weights for potentially faster inference

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading best model from: dse_results/rank_4_alpha_4


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [24]:
import torch.utils.data as data_utils

# Check evaluation accuracy
testset = load_dataset('ag_news', split='test')

tokenized_testset = testset.map(preprocess, batched=True,  remove_columns=["text"])
tokenized_testset = tokenized_testset.rename_column("label", "labels")
indices = torch.arange(1280)
tokenized_testset_sub = data_utils.Subset(tokenized_testset, indices)

_, _ = evaluate_model(inference_model, tokenized_testset_sub, True, 64, data_collator)

100%|██████████| 20/20 [00:12<00:00,  1.64it/s]

Evaluation Metric: {'accuracy': 0.86875}


In [22]:
#Load your unlabelled data
unlabelled_dataset = pd.read_pickle("test_unlabelled.pkl")
test_dataset = unlabelled_dataset.map(preprocess, batched=True, remove_columns=["text"])
unlabelled_dataset

Map: 100%|██████████| 8000/8000 [00:02<00:00, 3234.79 examples/s]


Dataset({
    features: ['text'],
    num_rows: 8000
})

In [25]:
# Run inference and save predictions
preds = evaluate_model(inference_model, test_dataset, False, 8, data_collator)
df_output = pd.DataFrame({
    'ID': range(len(preds)),
    'Label': preds.numpy()  # or preds.tolist()
})
df_output.to_csv(os.path.join(model_path,"inference_output.csv"), index=False)
print("Inference complete. Predictions saved to inference_output.csv")

100%|██████████| 1000/1000 [01:40<00:00,  9.90it/s]

Inference complete. Predictions saved to inference_output.csv
